In [ ]:
# CODE FOR LOADING SAVED CHATBOT
# import libraries
import random
import json
import pickle
import numpy as np
import nltk
import speech_recognition as sr
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob


# load intents and saved model output files
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl', 'rb'))
labels = pickle.load(open('labels.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

# separate input words from input sentence
def clean_up_sentences(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
	return sentence_words

# determine if input words in saved model words list
def bagw(sentence):
	sentence_words = clean_up_sentences(sentence)
	bag = [0]*len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

# predict label of sentence based on input words
# update: leveraging list comprehensions and dictionaries to work with confidence scores
def predict_class(sentence):
    bow = bagw(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [{'intent': labels[i], 'probability': str(res[i])}
               for i in range(len(res)) if res[i] > ERROR_THRESHOLD]
    results.sort(key=lambda x: x['probability'], reverse=True)
    return results

# print random response from appropriate responses for label
# if predicted intent has a low confidence score, gives a secondary response
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    probability = float(intents_list[0]['probability'])
    list_of_intents = intents_json['intents']
    result = ""
    if probability < 0.2:  # Adjust the threshold as needed
        result = "I'm sorry, but I'm not sure how to help with that. Could you please provide more information?"
    else:
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
    return result

# Capture the user's speech input
def listen_to_user():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = r.listen(source, timeout=10, phrase_time_limit=10)
            voice_input = r.recognize_google(audio)
            print("You said: {}".format(voice_input))
            return voice_input
        except sr.WaitTimeoutError:
            print("No command received in 10 seconds.")
            return ""
        except sr.UnknownValueError:
            print("Sorry, I didn't get that. Could you say it again?")
            return listen_to_user()
        except sr.RequestError:
            print("Sorry, my speech service is down. Please try again later.")
            return ""
        
# Initialize a variable to store the last bot reply
last_bot_reply = ""

# Initialize counts and list to store negative conversations
positive_count = 0
negative_count = 0
neutral_count = 0
total_count = 0
negative_conversations = []

# Determine sentiment of user's reply
def determine_sentiment(message):
    global positive_count, negative_count, neutral_count, total_count
    analysis = TextBlob(message)
    total_count += 1
    sentiment = "neutral"
    
    if analysis.sentiment.polarity > 0:
        positive_count += 1
        sentiment = 'positive'
    elif analysis.sentiment.polarity < 0:
        negative_count += 1
        sentiment = 'negative'
    else:
        neutral_count += 1
    
    return sentiment


# Determine user input method
def chat_input_method():
    print("How would you like to communicate with the bot?")
    print("1. Type\n2. Speak")
    method = input("Enter the number associated with your preferred method: ")
    
    if method == "1":
        user_input = input("You: ")
        print("You said: ", user_input)
        return user_input
    elif method == "2":
        return listen_to_user()
    else:
        print("Invalid input. Please enter 1 for Typing or 2 for Speaking.")
        return chat_input_method()

# create an infinite loop that prompts for input
print("Chatbot live. Say or type 'end' to stop the conversation.")
while True:
    message = chat_input_method()
    if message.lower() == "end":
        print("Bot: Goodbye! Have a nice day!")
        break
    elif message != "":
        ints = predict_class(message)
        res = get_response(ints, intents)

        sentiment = determine_sentiment(message)
        print(f"Sentiment: {sentiment}")
        # Calculate sentiment percentages
        if total_count > 0:
            print(f"Positive: {(positive_count/total_count)*100}%")
            print(f"Negative: {(negative_count/total_count)*100}%")
            print(f"Neutral: {(neutral_count/total_count)*100}%")
        
        if sentiment == 'negative' and (negative_count/total_count)*100 >= 50:
            print("Storing negative conversation for review.")
            with open('negative_conversations.txt', 'a') as f:
                f.write(f"Bot: {last_bot_reply}\n")
                f.write(f"User: {message}\n")
                f.write("---\n")

        # Overwrite the last_bot_reply with the current response of the bot
        last_bot_reply = res


        if res.startswith("I'm sorry"):
            print("Bot:", res)
        else:
            print("Bot:", res)


2023-12-11 13:49:21.255033: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 13:49:21.427285: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 13:49:21.427846: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 13:49:22.231673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-11 13:49:24.844214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

Chatbot live. Say or type 'end' to stop the conversation.
How would you like to communicate with the bot?
1. Type
2. Speak


Enter the number associated with your preferred method:  1
You:  stupid bot


You said:  stupid bot
1/1 [==============================] - 0s 52ms/step
Sentiment: negative
Positive: 0.0%
Negative: 100.0%
Neutral: 0.0%
Storing negative conversation for review.
Bot: Hi there! How can I help you?
How would you like to communicate with the bot?
1. Type
2. Speak


Enter the number associated with your preferred method:  1
You:  stupid bot you are amazing


...

You said:  thank you 
1/1 [==============================] - 0s 10ms/step
Sentiment: neutral
Positive: 40.0%
Negative: 40.0%
Neutral: 20.0%
Bot: Glad to help!
How would you like to communicate with the bot?
1. Type
2. Speak


Enter the number associated with your preferred method:  1
You:  I love you


You said:  I love you
1/1 [==============================] - 0s 9ms/step
Sentiment: positive
Positive: 50.0%
Negative: 33.33333333333333%
Neutral: 16.666666666666664%
Bot: No problem!
How would you like to communicate with the bot?
1. Type
2. Speak


Enter the number associated with your preferred method:  1
You:  stupid bot


You said:  stupid bot
1/1 [==============================] - 0s 10ms/step
Sentiment: negative
Positive: 42.857142857142854%
Negative: 42.857142857142854%
Neutral: 14.285714285714285%
Bot: Hello!
How would you like to communicate with the bot?
1. Type
2. Speak
